# Hands-On Deep Learning modelo CNN para series de tiempo
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-advanced-dl/blob/main/Unidad%201%20-%20Time%20Series/cnn-time-series.ipynb)

En este notebook vamos a usar capas CNN o Convotional Neural Networks para casos de series de tiempo. Vamos a continuar con el mismo dataset de la lección anterior con el fin de tener una visión holística de como se comparan las diferentes técnicas que hemos visto hasta ahora para el mismo problema. 

## Serie de tiempo del clima del laboratorio Max Planck
Para este notebook vamos a trabajar sobre una serie de datos climáticos cuyo dataset pueden encontrar aquí: https://www.kaggle.com/datasets/arashnic/max-planck-weather-dataset/data. Debido a que el dataset son 42MB y requiere autenticación en kaggle para descargar, esto deberán hacerlo manualmente antes de continuar con el notebook.

El objetivo de este caso es modelar la serie de tiempo de la temperatura atmosférica en grados centigrados. Para ello, contamos no solo con datos desde el 2009 al 2016 sobre la temperatura misma sino de otras mediciones como la presión atmosférica, presión de vapor, humedad, entre otros. Los datos fueron tomados especificamente en la estación del clima Beuternberg

### Enlaces de interés
- [Max-Planck Institut](https://mpimet.mpg.de/en/research/observations)
- [Max-Planck Institut - Weather Station](https://www.bgc-jena.mpg.de/wetter/)
- [Weather Station Beutenberg](https://www.bgc-jena.mpg.de/wetter/towercam.html)
- [Beutenberg Campus](https://maps.app.goo.gl/fGJX1T9bjTJhueqJ9)

In [1]:
import pkg_resources
import warnings
import os

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

# Para guardar modelos entrenados y re-utilizarlos luego
models_path = os.path.join(os.getcwd(), 'models')
os.makedirs(models_path, exist_ok=True)

In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-advanced-dl/raw/main/requirements.txt  && pip install -r requirements.txt

Primero, vamos a observar los datos y hacernos una idea de con que estamos lidiando

In [3]:
import pandas as pd
import numpy as np

# El dataset cuenta con temperatura en grados Kelvin también
# Pero no queremos incluir esta columna porque está midiendo exactamente lo mismo
columns = ['T (degC)', 'p (mbar)', 'VPact (mbar)', 'sh (g/kg)', 'Tdew (degC)', 'H2OC (mmol/mol)']
# Cambiar según la localización del archivo.
dataset = pd.read_csv('../datasets/max_planck_weather_ts.csv')[columns]

dataset.head(15)

,T (degC),p (mbar),VPact (mbar),sh (g/kg),Tdew (degC),H2OC (mmol/mol)
0,-8.02,996.52,3.11,1.94,-8.90,3.12
1,-8.41,996.57,3.02,1.89,-9.28,3.03
2,-8.51,996.53,3.01,1.88,-9.31,3.02
3,-8.31,996.51,3.07,1.92,-9.07,3.08
4,-8.27,996.51,3.08,1.92,-9.04,3.09
5,-8.05,996.50,3.14,1.96,-8.78,3.15
6,-7.62,996.50,3.26,2.04,-8.30,3.27
7,-7.62,996.50,3.25,2.03,-8.36,3.26
8,-7.91,996.50,3.15,1.97,-8.73,3.16
9,-8.43,996.53,3.00,1.88,-9.34,3.02


## Train-test split
Como este es un experimento un poco más realista, vamos a separar los conjuntos de datos en train y test. Como ya debemos saber, el conjunto de entrenamiento es exclusivamente para ese fin y el conjunto de prueba únicamente debemos usarlo para validar el modelo ante información no vista durante el entrenamiento y así poner a prueba su capacidad predictiva.

Como esto es una serie de tiempo, no podemos hacer un random sample ya que romperíamos la estructura secuencial de los datos. Asi que vamos a hacer algo más simple y dividir el conjunto en un punto particular. Vamos a trabajar con un train set correspondiente al 90% del conjunto original.

In [4]:
train_chunk = int(dataset.shape[0] * 0.9)

# Solo nos interesa trabajar con estas columnas por ahora
columns = ['T (degC)', 'p (mbar)', 'Tdew (degC)', 'H2OC (mmol/mol)']

# Necesitamos trabajar solo con datos informados, por lo que debemos descartar registros que no los tengan
train_set = dataset.iloc[:train_chunk][columns].dropna()
test_set = dataset.iloc[train_chunk:][columns].dropna()

## Baseline - Modelo SARIMAX
Preparamos un modelo SARIMAX como en la lección anterior para tenerlo como comparativa.

In [101]:
%%time
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarimax_model_path = os.path.join(models_path, 'sarimax_model.pkl')
relative_path = os.path.relpath(sarimax_model_path, os.getcwd())
if os.path.exists(sarimax_model_path):
    with open(sarimax_model_path, 'rb') as f:
        sarimax_model = pickle.load(f)
    
    print(f'Usando un modelo pre-entrenado en {relative_path}')
else:
    # Trabajaremos con los últimos 200k pasos de tiempo solamente.
    endog = train_set.loc[len(train_set)-200000:, 'T (degC)']
    exog = train_set.loc[len(train_set)-200000:, ['p (mbar)', 'Tdew (degC)', 'H2OC (mmol/mol)']].values

    sarimax_model = SARIMAX(endog, exog, order=(1, 1, 2), seasonal_order=(1, 1, 2, 4), simple_differencing=False)
    res = sarimax_model.fit(disp=False)

    with open(sarimax_model_path, 'wb') as f:
        pickle.dump(sarimax_model, f)

    print(f'Guardando modelo en {relative_path}')

res = sarimax_model.fit(disp=False)
print(res.summary())

Usando un modelo pre-entrenado en models/sarimax_model.pkl
                                     SARIMAX Results                                     
Dep. Variable:                          T (degC)   No. Observations:               200000
Model:             SARIMAX(1, 1, 2)x(1, 1, 2, 4)   Log Likelihood               46387.378
Date:                           Sat, 02 Mar 2024   AIC                         -92754.757
Time:                                   12:23:07   BIC                         -92652.696
Sample:                                        0   HQIC                        -92724.718
                                        - 200000                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2991   3.35e-06  -8.93e+04      0.000      -0.

In [6]:
from sklearn.metrics import mean_squared_error

test_endog = test_set['T (degC)']
test_exog = test_set[['p (mbar)', 'Tdew (degC)', 'H2OC (mmol/mol)']]

sarimax_preds = res.get_forecast(steps=test_endog.shape[0], exog=test_exog).predicted_mean

validation = pd.DataFrame({'data': test_endog.values, 'prediction': sarimax_preds.values})

mse = mean_squared_error(validation['data'], validation['prediction'])
print(f"MSE Modelo SARIMAX: {mse:.4f}")

MSE Modelo SARIMAX: 75.0673


## Modelo CNN simple

![](../assets/TS-SimpleLSTM.drawio-wbg.png)

En este caso vamos a trabajar dos versiones diferentes de un modelo CNN simple para este caso. En el primero, vamos a usar convoluciones 2D, para lo cual necesitaremos hacer algunos ajustes a los datos mientras pasan por la red. Luego, vamos a usar un modelo con convoluciones 1D, donde los datos que tenemos se facilitan más para este caso.

In [7]:
%%time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pytorch_lightning import LightningModule, Trainer
from typing import List, Tuple
from torch.nn import functional as F
from lightning.pytorch.loggers import TensorBoardLogger


class TimeSeriesDataset(Dataset):
    """Time Series Dataset.
    Define un pytorch dataset desde un pandas dataframe.
    Asume que el dataframe provisto es una serie donde
    cada fila corresponde a un paso de tiempo t y cada
    columna son el vector a procesar.
    """
    def __init__(self, 
                 dataframe: pd.DataFrame,
                 target_col: str,
                 feature_cols:  List[str],
                 sequence_length: int = 10) -> None:
        super().__init__()
        self._dataframe = dataframe.reset_index(drop=True)
        self.sequence_length = sequence_length
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.scaler = StandardScaler()

    
    def __len__(self):
        # Porque a partir de n - sequence length, no se podrán extraer más secuencias.
        return len(self._dataframe) - self.sequence_length
    
    def __getitem__(self, index) -> Tuple[torch.Tensor]:
        start_idx = index
        end_idx = index + self.sequence_length - 1

        # Extraemos las secuencias, el valor objetivo debe estar incluido en la secuencia.
        sequence = self._dataframe.loc[start_idx:end_idx, [self.target_col] + self.feature_cols].values
        # Aplicamos escala si es necesario
        sequence = self.scaler.fit_transform(sequence)
        
        # Extraemos el objetivo
        target = self._dataframe.loc[end_idx, self.target_col]

        # Convertimos a tensores de pytorch
        sequence = torch.tensor(sequence, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)

        return sequence, target


sequence_length = 200
train_dataset = TimeSeriesDataset(
    train_set[columns], 
    target_col='T (degC)', 
    feature_cols=['p (mbar)', 'Tdew (degC)', 'H2OC (mmol/mol)'],
    sequence_length = sequence_length
    )

train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, train_size=0.8)

# Creamos los dataloaders para el entrenamiento de la red
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=6)
val_loader = DataLoader(val_dataset, batch_size=128, num_workers=6)
    

CPU times: user 11min 45s, sys: 2.1 s, total: 11min 47s
Wall time: 11min 47s


### Definición del modelo y entrenamiento

Ahora vamos a definir un modelo convolucional 2D. Recordemos que en redes convolucionales hay que ser cuidadosos con las dimensiones de entrada y salida de las capas ya que estas se van cambiando, en función de algunos hiper-parámetros, a medida que van pasando por las capas. Siempre tener presente las siguiente formulas:

#### Dimensiones de salida de capa convolucional y pooling
$$
W_{out} = \frac{W_{in} - K + 2P}{S} + 1 \\
H_{out} = \frac{H_{in} - K + 2P}{S} + 1
$$

Donde: 
- $W$ y $H$, en un tensor de rango 2, representarían el largo y ancho del mismo (si fuera una imágen), para tensores de rango 1, es único vector que lo compone.
- $K$ es el tamaño del kernel que se usa en la capa
- $P$ es el tamaño del padding definido. Cuando especificamos un padding tipo `same`, quiere decir que vamos a agregar padding hasta que el output tenga el mismo shape que el input.
- $S$ es el stride of desplazamiento del kernel

Por ejemplo, si esperamos un tensor de entrada de $200 \times 4$ y definimos una capa convolucional 2D, con `kernel_size=4`, `padding=1` y `stride=1`, el output shape sería:
$$
W_{out} = \frac{200 - 4 + 2 \cdot 1}{1} + 1 = 199 
$$


In [8]:
%%time
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

class SimpleCNN2D(LightningModule):

    def __init__(self):
        super(SimpleCNN2D, self).__init__()

        # entra (1, 200, 4) => (un canal, longitud de la secuencia, features del time step)
        self.cnn_block = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=4, padding='same', padding_mode='replicate', stride=1),  # sale (16, 200, 4)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=1),  # sale (16, 101, 3)
            nn.Conv2d(16, 64, kernel_size=3, padding='same', stride=1),  # sale (64, 101, 3)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),  # sale (64, 33, 1)
            nn.Flatten(start_dim=1)  # sale (2112)
        )

        self.fc_block = nn.Sequential(
            nn.Linear(2112, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        self.hparams['cnn_block'] = str(self.cnn_block)
        self.hparams['fc_block'] = str(self.fc_block)
        self.save_hyperparameters()

    def forward(self, x):
        # Como nuestro dato de entrada solo tiene un valor por cada par tiempo-feature y
        # Las capas convolucionales esperan un tensor de tercer rango, debemos
        # expandir el rango del tensor, entonces pasamos de
        # (batch, sequence_length, feature_dim) a (batch, channel, sequence_length, feature_dim)
        # aquí channel es un placeholder para reservar el espacio para esa dimension.
        x = x.unsqueeze(1)
        x = self.cnn_block(x)
        return self.fc_block(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.unsqueeze(-1))
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.unsqueeze(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.unsqueeze(-1))
        self.log('test_loss', loss)

    def predict_step(self, batch):
        x, y = batch
        return torch.cat([self(x), y.unsqueeze(-1)], axis=-1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

cnn2d_model = SimpleCNN2D()

tb_logger = TensorBoardLogger('tb_logs', name='SimpleCNN2D')
trainer2d = Trainer(max_epochs=10, devices=1, logger=tb_logger, callbacks=[EarlyStopping(monitor='val_loss', mode='min')])
trainer2d.fit(cnn2d_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | cnn_block | Sequential | 9.6 K 
1 | fc_block  | Sequential | 557 K 
-----------------------------------------
566 K     Trainable params
0         Non-trainable params
566 K     Total params
2.268     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 2365/2365 [00:17<00:00, 135.69it/s, v_num=3, train_loss=3.680]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 2365/2365 [00:17<00:00, 135.52it/s, v_num=3, train_loss=3.680]
CPU times: user 2min 28s, sys: 27.7 s, total: 2min 55s
Wall time: 2min 57s


Finalizado el entrenamiento, procedemos a validar los resultados en el conjunto de prueba.

In [9]:
cnn2d_model.eval()
test_dataset = TimeSeriesDataset(
    test_set[columns], 
    target_col='T (degC)', 
    feature_cols=['p (mbar)', 'Tdew (degC)', 'H2OC (mmol/mol)'],
    sequence_length = sequence_length
    )
test_loader = DataLoader(test_dataset, batch_size=512)
result2d = trainer2d.test(dataloaders=test_loader)

Restoring states from the checkpoint path at tb_logs/SimpleCNN2D/version_3/checkpoints/epoch=9-step=23650.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at tb_logs/SimpleCNN2D/version_3/checkpoints/epoch=9-step=23650.ckpt


Testing DataLoader 0: 100%|██████████| 82/82 [00:30<00:00,  2.70it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            33.35602569580078
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


#### Definición de modelo convolucional 1D

Ahora vamos a definir un segundo modelo, esta vez utilizando capas convolucionales 1D. Las reglas de cálculo para el output shape siguen siendo las mismas, sin embargo la forma como las operamos y pasamos el input es ligeramente distinta. El resto del modelo y entrenamiento siguen siendo los mismos.

In [10]:
%%time
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

class SimpleCNN1D(LightningModule):

    def __init__(self):
        super(SimpleCNN1D, self).__init__()

        # entra (4, 200) => (un canal, longitud de la secuencia)
        self.cnn_block = nn.Sequential(
            nn.Conv1d(in_channels=4, out_channels=16, kernel_size=3, stride=1),  # sale (16, 198)
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3),  # sale (16, 66)
            nn.ReLU(),
            nn.Conv1d(16, 64, kernel_size=3, stride=1),  # sale (64, 64)
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3),  # sale (64, 21)
            nn.Flatten(start_dim=1)  # sale (1344)
        )

        self.fc_block = nn.Sequential(
            nn.Linear(1344, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        self.hparams['cnn_block'] = str(self.cnn_block)
        self.hparams['fc_block'] = str(self.fc_block)
        self.save_hyperparameters()

    def forward(self, x):
        # Debemos intercambiar las dimensiones del tensor de entrada
        # Recordemos que entra (batch, sequence_length, feature_dim)
        # Las capas Conv1D experan recibir la secuencia en la última dimension
        # ya que los kernels se van a desplazar en esta dirección
        x = x.permute(0, 2, 1)
        x = self.cnn_block(x)
        return self.fc_block(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.unsqueeze(-1))
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.unsqueeze(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.unsqueeze(-1))
        self.log('test_loss', loss)

    def predict_step(self, batch):
        x, y = batch
        return torch.cat([self(x), y.unsqueeze(-1)], axis=-1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

cnn1d_model = SimpleCNN1D()

tb_logger = TensorBoardLogger('tb_logs', name='SimpleCNN1D')
trainer1d = Trainer(max_epochs=10, devices=1, logger=tb_logger, callbacks=[EarlyStopping(monitor='val_loss', mode='min')])
trainer1d.fit(cnn1d_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | cnn_block | Sequential | 3.3 K 
1 | fc_block  | Sequential | 360 K 
-----------------------------------------
364 K     Trainable params
0         Non-trainable params
364 K     Total params
1.457     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 9: 100%|██████████| 2365/2365 [00:17<00:00, 132.74it/s, v_num=3, train_loss=3.020]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 2365/2365 [00:17<00:00, 132.61it/s, v_num=3, train_loss=3.020]
CPU times: user 2min 29s, sys: 26.7 s, total: 2min 56s
Wall time: 2min 58s


Finalizado el entrenamiento procedemos a validar en el conjunto de prueba

In [11]:
cnn1d_model.eval()
test_loader = DataLoader(test_dataset, batch_size=512)
result1d = trainer1d.test(dataloaders=test_loader)

Restoring states from the checkpoint path at tb_logs/SimpleCNN1D/version_3/checkpoints/epoch=9-step=23650.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at tb_logs/SimpleCNN1D/version_3/checkpoints/epoch=9-step=23650.ckpt


Testing DataLoader 0: 100%|██████████| 82/82 [00:30<00:00,  2.68it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            36.12657928466797
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Ahora comparemos las curvas de aprendizaje de los modelos que hemos entrenado hasta ahora.

In [12]:
%load_ext tensorboard

In [13]:
%tensorboard --logdir tb_logs/

### Resultados interesantes

Observamos que en comparación con la pérdida en entrenamiento esta es significativamente mayor, lo cual sugiere que el modelo está memorizando los datos, es decir, estaría haciendo overfitting. Ya sabemos que esto lo podríamos tratar con regularización en la red y/o en la función de pérdida.

También, desde el punto de vista de la comparación con los otros modelos, todos los modelos parecen ser superiores al modelo clásico en métricas. Pero recordemos siempre hacer una comprobación visual.

| Modelo | MSE Loss |
|--------|----------|
| SARIMAX| 75.063  |
| MLP    | 32.8617  |
| LSTM   | 30.6136  |
| CNN-2d   | 33.3560  |
| CNN-1d   | 36.1265  |

## Pronósticos

Ya que tenemos varios modelos entrenados, podemos hacer comparaciones entre estos y ver cuales se ajustan mejor a los datos. Para ello vamos a cargar los modelos pre-entrenados que tenemos (estarán disponibles si se han ejecutado los notebooks anterioes y/o se han subido los archivos necesarios en las rutas especificadas)

In [98]:
from time_series_models import SimpleMLP, SimpleLSTM

steps = 2000
# Por defecto se cargarán los últimos modelos en sus respectivos directorios.
# Pueden modificar las rutas a un modelo fijo especifico si lo desean.
models = {
    "mlp": os.path.join("tb_logs/SimpleMLP", sorted(os.listdir('tb_logs/SimpleMLP/'))[-1]),
    "lstm": os.path.join("tb_logs/SimpleLSTM", sorted(os.listdir('tb_logs/SimpleLSTM/'))[-1]),
    "cnn1d": os.path.join("tb_logs/SimpleCNN1D", sorted(os.listdir('tb_logs/SimpleCNN1D/'))[-1]),
    "cnn2d": os.path.join("tb_logs/SimpleCNN2D", sorted(os.listdir('tb_logs/SimpleCNN2D/'))[-1]),
}

predictions = {
    "sarimax": sarimax_preds[-steps-10:][:-10],
}

added_ground_truth = False

for name, path in models.items():
    cp_path = os.path.join(path, 'checkpoints')
    cp_file = os.path.join(cp_path, os.listdir(cp_path)[0])
    relative_path = os.path.relpath(cp_file, os.getcwd())
    print(f"Usando {cp_file}")
    if name == "mlp":
        model = SimpleMLP.load_from_checkpoint(cp_file)
    elif name == "lstm":
        model = SimpleLSTM.load_from_checkpoint(cp_file)
    elif name == "cnn1d":
        model = SimpleCNN1D.load_from_checkpoint(cp_file)
    elif name == "cnn2d":
        model = SimpleCNN2D.load_from_checkpoint(cp_file)
    else:
        raise ValueError(f"Modelo no válido: {name}")

    trainer = Trainer()
    batched_preds = trainer.predict(model, dataloaders=test_loader)
    preds = np.concatenate([b.numpy() for b in batched_preds])
    predictions[name] = preds[-steps:, 0]

    if not added_ground_truth:
        predictions['ground_truth'] = preds[-steps:, 1]
        added_ground_truth = True
    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Usando tb_logs/SimpleMLP/version_1/checkpoints/epoch=9-step=23650.ckpt
Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting DataLoader 0: 100%|██████████| 82/82 [00:31<00:00,  2.63it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Usando tb_logs/SimpleLSTM/version_9/checkpoints/epoch=9-step=23650.ckpt
Predicting DataLoader 0: 100%|██████████| 82/82 [00:33<00:00,  2.44it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Usando tb_logs/SimpleCNN1D/version_3/checkpoints/epoch=9-step=23650.ckpt
Predicting DataLoader 0: 100%|██████████| 82/82 [00:31<00:00,  2.61it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Usando tb_logs/SimpleCNN2D/version_3/checkpoints/epoch=9-step=23650.ckpt
Predicting DataLoader 0: 100%|██████████| 82/82 [00:30<00:00,  2.67it/s]


In [99]:
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [100]:
from collections import defaultdict
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource


bokeh_data = defaultdict(list)
for column in visual_check.columns:
    bokeh_data['x'].append(visual_check.index.values)
    bokeh_data['series'].append(visual_check[column].values)
    bokeh_data['name'].append(column)

bokeh_data['color'] = Category10[6]

source = ColumnDataSource(bokeh_data)

TOOLS = "pan,box_zoom,reset,save,hover,tap"
TOOLTIPS=[
    ("Serie", "@name"),
    ("Temperatura", "$y{0.0 a}°")
]
p = figure(
    title='Comparación visual entre pronósticos y realidad', 
    tools=TOOLS,
    tooltips=TOOLTIPS,
    width_policy='max'
    )

alpha_values = [1.0 if col == 'ground_truth' else 0.3 for col in visual_check.columns]
p.multi_line(
    xs='x',
    ys='series',
    line_color='color',
    line_alpha=0.6,
    hover_line_alpha=1.0,
    line_width=2,
    hover_line_width=3,
    legend_field='name',
    source=source
)


p.yaxis.axis_label = "Temperatura"
p.xaxis.axis_label = "Tiempo"

show(p)


Observamos que en geneal cada modelo tiene varios aciertos y errores. Algo interesante que vale la pena seguir resaltando es que si bien todos los modelos de DL tienen mejores métricas en general, al graficarlos todos tiene un comportamiento errático en comparación con el modelo SARIMAX. Esto puede deberse a múltiples factores y quizás se pueda mejorar haciendo ajustes a los modelos. Sin embargo, los modelos de DL tienden a capturar mejor los altibajos de de la serie, solo que con una magnitud mayor a la esperada.

La elección del mejor modelo debe basarse en múltiples factores. Para algunos casos la precisión es un aspecto no negociable pero por otros lados, lograr una alta precisión puede ser una tarea "imposible" por factores incluso externos a la información de la que disponemos. Aquí por ejemplo es donde ensambles de modelos pueden llegar a ser útiles para llegar a un consenso sobre las predicciones de los modelos. También hay otros factores a considerar como la simplicidad y la cantidad de recursos necesarios para modelar. No en todas las ocasiones se puede o debe usar un modelo clásico e igualmente un modelo de DL.

Un detalle final para este ejercicio es que se ha trabajado con arquitecturas simples y genéricas, dando libertad de implenetación a quien estudie el problema. En la siguiente lección vamos a explorar una arquitectura especifica para series de tiempo y vamos a poder cerrar el ciclo de conocimientos en el tema.